参考 https://aws.amazon.com/cn/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/

In [3]:
cd code

/home/ec2-user/SageMaker/code


In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

import os
import numpy as np
import skimage as sk
import skimage.io
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from online_training import *


PATH        = '../data/dogfacenet/' 
PATH_SAVE   = '../output/history/'  
PATH_MODEL  = './' 
SIZE        = (224,224,3)    
TEST_SPLIT  = 0.1   

LOAD_NET    = True 
NET_NAME    = '2019.05.12.dogfacenet' 
START_EPOCH = 7  
NBOF_EPOCHS = 250 
STEPS_PER_EPOCH = 300
VALIDATION_STEPS = 30


alpha = 0.3
def triplet(y_true,y_pred):
    
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)

    return K.sum(tf.nn.relu(ap - an + alpha))

def triplet_acc(y_true,y_pred):
    a = y_pred[0::3]
    p = y_pred[1::3]
    n = y_pred[2::3]
    
    ap = K.sum(K.square(a-p),-1)
    an = K.sum(K.square(a-n),-1)
    
    return K.less(ap+alpha,an)

with tf.Session() as sess: 

    model = tf.keras.models.load_model(
        '{:s}{:s}.{:d}.h5'.format(PATH_MODEL,NET_NAME,START_EPOCH),
        custom_objects={'triplet':triplet,'triplet_acc':triplet_acc})

    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    !rm -rf export
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    builder = builder.SavedModelBuilder(export_dir)
    signature = predict_signature_def(
        inputs={"inputs": model.input}, outputs={"score": model.output})


    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()


INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [6]:
ls export/Servo/1

saved_model.pb  variables/


In [7]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [28]:
!touch train.py
import boto3, re
from sagemaker import get_execution_role
role = get_execution_role()
from sagemaker.tensorflow.serving import Model
sagemaker_model = Model(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version='1.12.0',
                                  entry_point = 'train.py')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [29]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!CPU times: user 5.69 s, sys: 626 ms, total: 6.31 s
Wall time: 6min 39s
